INTEGRANTES - RM

Caiky de Ávila Pereira Dutra                 99274    

Giovanna Shiguemori Barbosa                  99105

Isabela Victoria de Novais Romanato          550234

Lucas Pinheiro de Souza                      99061

Yann Dantas                                  550783


In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [120]:
df = pd.read_csv('/content/drive/MyDrive/datasets/games.csv')

**Em nenhuma hipótese a base poderá ficar com menos de 10.000 registros de trabalho**

In [121]:
df.shape

(85103, 39)

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85103 entries, 0 to 85102
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       85103 non-null  int64  
 1   Name                        85097 non-null  object 
 2   Release date                85103 non-null  object 
 3   Estimated owners            85103 non-null  object 
 4   Peak CCU                    85103 non-null  int64  
 5   Required age                85103 non-null  int64  
 6   Price                       85103 non-null  float64
 7   DLC count                   85103 non-null  int64  
 8   About the game              81536 non-null  object 
 9   Supported languages         85103 non-null  object 
 10  Full audio languages        85103 non-null  object 
 11  Reviews                     9743 non-null   object 
 12  Header image                85103 non-null  object 
 13  Website                     394

In [123]:
# CORRELAÇÃO DE VARIÁVEIS NUMÉRICAS

corr_matrix = df[['Price', 'Release date', 'Peak CCU', 'Required age', 'Metacritic score', 'Positive', 'Negative', 'Average playtime forever', 'Average playtime two weeks', 'Median playtime forever', 'Median playtime two weeks']].corr()

print(corr_matrix['Price'].sort_values(ascending=False))


Price                         1.000000
Metacritic score              0.141263
Required age                  0.096285
Average playtime two weeks    0.065388
Median playtime two weeks     0.064112
Average playtime forever      0.062672
Median playtime forever       0.041303
Peak CCU                      0.039737
Positive                      0.029271
Negative                      0.025202
Name: Price, dtype: float64


<ipython-input-123-132944f9b988>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df[['Price', 'Release date', 'Peak CCU', 'Required age', 'Metacritic score', 'Positive', 'Negative', 'Average playtime forever', 'Average playtime two weeks', 'Median playtime forever', 'Median playtime two weeks']].corr()


**Utilizar somente registro com "Price" maior que zero e "Median playtime forever"maior que 60.**

In [124]:
# tratamento variável categórica
df['year'] = pd.to_datetime(df['Release date']).apply(lambda x:1 if x.year > 2000 else 0)

df = df[df['Price'] > 0]
df = df[df['Median playtime forever'] > 60]

**Explicar a variável "Price" utilizando qualquer outra variável existente no conjunto fornecido.**


In [125]:
# Regressão linear
X = df[['year', 'Required age', 'Metacritic score', 'Negative', 'Average playtime two weeks',
        'Median playtime two weeks','Peak CCU', 'Average playtime forever', 'Positive', 'Median playtime forever']]
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Regressão polinomial
poly_features = PolynomialFeatures(degree=2)
X_poly = poly_features.fit_transform(X)

X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X_poly, y, test_size=0.2, random_state=42)

model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train_poly)

y_pred_poly = model_poly.predict(X_test_poly)

**A Regressão Linear deve apresentar um R-quadrado ajustado de ao menos 0,15
(15%).**


In [126]:
X_train_sm = sm.add_constant(X_train)
model_sm = sm.OLS(y_train, X_train_sm)
results = model_sm.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     111.2
Date:                Mon, 01 Apr 2024   Prob (F-statistic):          1.01e-217
Time:                        17:50:03   Log-Likelihood:                -31846.
No. Observations:                8219   AIC:                         6.371e+04
Df Residuals:                    8208   BIC:                         6.379e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

**Comparar os resultados da Regressão Linear e de uma Regressão Polinomial de grau 2. Escreva um parágrafo indicando qual foi o melhor resultado encontrado e por quê.**

In [127]:
# Regressão Linear
mse_linear = np.mean((y_test - y_pred)**2)

# Regressão Polinomial
mse_poly = np.mean((y_test_poly - y_pred_poly)**2)

# Escreva um parágrafo indicando qual foi o melhor resultado encontrado e por quê.

if mse_linear < mse_poly:
  print("A regressão linear obteve o melhor resultado, com um MSE de", mse_linear, "em comparação com", mse_poly, "da regressão polinomial.")
  print("Isso sugere que um modelo linear é mais adequado para esses dados.")
else:
  print("A regressão polinomial obteve o melhor resultado, com um MSE de", mse_poly, "em comparação com", mse_linear, "da regressão linear.")
  print("Isso sugere que um modelo polinomial de grau 2 é mais adequado para esses dados.")


A regressão polinomial obteve o melhor resultado, com um MSE de 125.22245713291883 em comparação com 129.69859967338954 da regressão linear.
Isso sugere que um modelo polinomial de grau 2 é mais adequado para esses dados.


**Utilize pelo menos 5.000 registros para a amostra de treino. Para o teste, verifique se
houve overfitting com significância alfa de 10% e ao menos 10 amostras.**
Pontos-bônus para utilização de Shapiro e Levene para o teste de hipótese necessário.

In [128]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train = df_train.sample(n=5000)

model = LinearRegression()
model.fit(df_train[['year']], df_train['Price'])

y_pred = model.predict(df_test[['year']])

f_statistic, p_value = stats.f_oneway(df_train['Price'], y_pred)
if p_value < 0.1 and len(df_test) >= 10:
  print("Overfitting detected", p_value)
else:
  print("No overfitting detected")

No overfitting detected


**No modelo treinado, é possível aumentar o grau da Regressão Polinomial? Explique.**

Sim, é possível aumentar o grau do modelo de regressão polinomial. No entanto, é importante notar que aumentar o grau do polinômio pode levar ao overfitting, que ocorre quando o modelo é muito complexo e começa a aprender o ruído nos dados em vez da relação subjacente entre as variáveis.

Portanto, é importante avaliar cuidadosamente o desempenho e a complexidade do modelo ao aumentar o grau do modelo de regressão polinomial. Uma maneira de fazer isso é usar a validação cruzada para avaliar o desempenho do modelo em dados não vistos.

Aqui está um exemplo de como aumentar o grau do modelo de regressão polinomial:


**Com as suas palavras, explique qual a diferença entre uma Regressão Linear e uma
Regressão Polinomial.**

A regressão linear e a regressão polinomial são técnicas utilizadas na análise estatística para modelar e prever a relação entre variáveis ​​independentes e dependentes. Na regressão linear, assume-se que essa relação é aproximadamente linear, ou seja, os dados seguem uma tendência linear. Por outro lado, na regressão polinomial, permite-se que a relação entre as variáveis seja modelada por uma função polinomial de grau maior que um, o que pode capturar relações mais complexas entre os dados. Ambas as técnicas são amplamente empregadas em várias áreas, incluindo ciências sociais, economia, engenharia e ciência dos dados, para análise e previsão de dados.

**Consideração e justificativa**

A partir do exercício realizado, é considerável que não atingimos um bom modelo. Analisamos as variáveis que poderiam influenciar para o 'Price', a principal categórica seria 'Release date', e utilizamos o exemplo em aula tratando os anos maiores que 2000. Para aumentar o R-quadrado ajustado, fizemos a correlação com as variáveis numéricas, utilizar as que mais relacionam deicaram o modelo pobre, então a alternativa foi utilizar as demais também.


